# Training with reduced data

In this notebook, we validate the working of our stabilization methods exposed to training with less data

# CIFAR10

In [5]:
import sys, time, torch
from utils.training.training_ops import Average, accuracy, loader_accuracy

# Make the training / testing loaders
from utils.training.datasets import get_cifar10
train_loader, test_loader = get_cifar10(batch_size=128, shuffle=False)
mu, std = [0.4914, 0.4822, 0.4465], [0.247, 0.243, 0.261]

# Define the model
from utils.solvers.fixed_step import FixedStepSolver, FixedStepRK4RegSolver, FixedStepNumericalLyapunovSolver, FixedStepNumericalLyapunovSolverV2
from utils.integrators.simple import Euler, ModifiedEuler, RungeKutta4, K2K3L1DistRungeKutta4, K2K3L2DistRungeKutta4, K2K3CosSimRungeKutta4
from utils.networks.odenet_2 import OdeNet

# Define experiments
experiments = [
    (Euler, FixedStepSolver, 2, None, None),
    (Euler, FixedStepNumericalLyapunovSolverV2, 2, 1e-3, 2),
    (K2K3L2DistRungeKutta4, FixedStepRK4RegSolver, 1, 1e-4, None),
]

# Define percentage of data used
data_perc = 0.01

for intr, sol, steps, le_reg, vectors in experiments:
    for i in range(3):
        # Define the model
        model = OdeNet(solver=sol, integrator=intr, t0=0.0, t1=1.0, steps=steps, in_channels=3, channels=[64, 128, 256, 512], classes=10, mu=mu, std=std)
        model.solver1.n, model.solver2.n, model.solver3.n, model.solver4.n = vectors, vectors, vectors, vectors

        # Naming for saving
        post_fix = "%sStep-eps1-leReg%s-Vecs%s-Run%s" % (str(steps), str(le_reg), str(vectors), str(i))
        print("\n" + model.solver1.name + "-" + model.solver1.method.name + "-" + post_fix)

        # Define the convenient average calculators
        time_keeper, train_acc, train_exps, test_acc, test_exps = Average(), Average(), Average(), Average(), Average()

        epochs = 20
        best_acc = 0
        for e in range(epochs):
            # Train loop
            model.train()
            for i, (data, target) in enumerate(train_loader):
                s = time.time()
                # Convert data proper device, forward pass and calculate loss
                data, target = data.to(model.device), target.to(model.device)
                pred = model(data)
                ce_loss = model.loss_module(pred, target)
                
                #Take optimizer step
                model.optimizer.zero_grad()
                if le_reg and model.exps:
                    (ce_loss + le_reg * model.exps).backward()
                else:
                    ce_loss.backward()
                model.optimizer.step()

                # Update metrics
                time_keeper.update(time.time() - s)
                train_acc.update(accuracy(pred, target))
                if le_reg and model.exps:
                    train_exps.update(model.exps.item())
                
                # Stop training early
                if i == int(len(train_loader) * data_perc):
                    break
            
            # Evaluation loop
            with torch.no_grad():
                for i, (data, target) in enumerate(test_loader):
                    data, target = data.to(model.device), target.to(model.device)
                    pred = model(data)
                    ce_loss = model.loss_module(pred, target)
                    test_acc.update(accuracy(pred, target))
                    if le_reg and model.exps:
                        test_exps.update(model.exps.item())
            print('Epoch: %d / %d | Time per iter %.3f | Train acc: %.3f | Train exps: %.3f | Test acc: %.3f | Test exps: %.3f' % (e + 1, epochs, time_keeper.eval(), 100 * train_acc.eval(), train_exps.eval(), 100 * test_acc.eval(), test_exps.eval()))

            # Reset statistics each epoch:
            time_keeper.reset(), train_acc.reset(), train_exps.reset(), test_acc.reset(), test_exps.reset()

            # Decay Learning Rate
            model.scheduler.step()

Files already downloaded and verified
Files already downloaded and verified
This solver will be using the following time deltas: [0.5, 0.5]
This solver will require 2 gradient evaluations
This model is using 7847946 parameters
This model is using 12 Conv layers

FixedStepSolver-Euler-2Step-eps1-leRegNone-VecsNone-Run0
Epoch: 1 / 20 | Time per iter 0.025 | Train acc: 13.867 | Train exps: 0.000 | Test acc: 17.548 | Test exps: 0.000
Epoch: 2 / 20 | Time per iter 0.026 | Train acc: 21.484 | Train exps: 0.000 | Test acc: 26.342 | Test exps: 0.000
Epoch: 3 / 20 | Time per iter 0.027 | Train acc: 25.586 | Train exps: 0.000 | Test acc: 22.696 | Test exps: 0.000
Epoch: 4 / 20 | Time per iter 0.025 | Train acc: 25.781 | Train exps: 0.000 | Test acc: 30.839 | Test exps: 0.000
Epoch: 5 / 20 | Time per iter 0.026 | Train acc: 35.742 | Train exps: 0.000 | Test acc: 31.801 | Test exps: 0.000
Epoch: 6 / 20 | Time per iter 0.026 | Train acc: 39.062 | Train exps: 0.000 | Test acc: 32.903 | Test exps: 0.